# Random Forest Regressor...

### Python Library ...

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense,Dropout
from tensorflow.keras.optimizers import Adam,SGD
from matplotlib import pyplot as plt


### Load All The Data....

In [3]:
# Load the data
file_path = '../New_Data/Final_Dataset.xlsx'  # Replace with your file path
data_01 = pd.read_excel(file_path)

### Creating Synthetic Data ....

In [4]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from tsaug import AddNoise, TimeWarp, Drift

# Original dataset (features only)
Data = data_01.loc[:, 'ROM_angle':90].values 

# Number of synthetic samples to generate
num_synthetic_samples = 20

# Extract Min and Max ROM angle from the original data
rom_angle_min = Data[:, 0].min()
rom_angle_max = Data[:, 0].max()

# Function to constrain ROM angle within original Min and Max range
def constrain_rom_angle(augmented_sample, rom_angle_min, rom_angle_max):
    augmented_sample[:, 0] = np.clip(augmented_sample[:, 0], rom_angle_min, rom_angle_max)
    return augmented_sample

# Augmentation pipeline
augmenter = (
    TimeWarp(n_speed_change=1, max_speed_ratio=1.2) +
    AddNoise(scale=0.005) +
    Drift(max_drift=0.01, n_drift_points=2)
)
# Generate synthetic samples
synthetic_samples = []
for _ in range(num_synthetic_samples):
    sampled_rows = Data[np.random.choice(Data.shape[0], size=1, replace=False)]
    augmented_sample = augmenter.augment(sampled_rows)
    augmented_sample = constrain_rom_angle(augmented_sample, rom_angle_min, rom_angle_max)
    synthetic_samples.append(augmented_sample)

# Combine original and synthetic samples
synthetic_samples = np.vstack(synthetic_samples)
augmented_dataset = np.vstack([Data, synthetic_samples])

data = pd.DataFrame(augmented_dataset)
data.head()


### Extract EEG features and target...

In [6]:
# Extract EEG features and target
X = data.loc[:, 1:181].values  # EEG data (columns D to MI) iloc[:,3:]
y = data.loc[:, 0].values  # Target ROM angle (column C)
max_rom = y.max()
min_rom = y.min()
# Check for extreme values in target
print(f"Max ROM Angle: {max_rom}, Min ROM Angle: {min_rom}")

# Normalise the target variable
y = (y - min_rom) / (max_rom - min_rom)


Max ROM Angle: 155.0, Min ROM Angle: 105.0


### Normalise the features...

In [7]:
# Normalise the features
scaler = MinMaxScaler()
X_normalised = scaler.fit_transform(X)

# Reshape X for GRU (samples, timesteps, features)
n_features = X_normalised.shape[1]
X_reshaped = X_normalised.reshape(X_normalised.shape[0], 1, n_features)

In [8]:
reshaped_array = X_reshaped.squeeze(axis=1)  # Remove the second dimension
print(reshaped_array.shape)

(127, 181)


### Split data into training and testing sets ...

In [9]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.1, random_state= 4)

In [10]:
import numpy as np

# Reshape to 2D
X_X = X_train.squeeze(axis=1)
X_Xtest = X_test.squeeze(axis=1)

print(X_X)

[[0.41946039 0.18590014 0.09169899 ... 0.12009779 0.11756412 0.0798004 ]
 [0.34401183 0.03094984 0.         ... 0.07936508 0.07277167 0.02661783]
 [0.8336619  0.80576307 0.82462892 ... 0.         0.         0.        ]
 ...
 [0.24172439 0.17503396 0.1543082  ... 0.         0.         0.        ]
 [0.25769353 0.19840308 0.17693137 ... 0.         0.         0.        ]
 [0.94502428 0.97502609 0.9975593  ... 0.         0.         0.        ]]


### Random Forest Regressor Model...

In [11]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_X, y_train)
predictions_RF = model.predict(X_Xtest)


### Final R2 Score...

In [12]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Make predictions
predictions_RF = model.predict(X_Xtest)

r2 = r2_score(y_test,predictions_RF)
mae = mean_absolute_error(y_test,predictions_RF)
mse = mean_squared_error(y_test,predictions_RF)

print(f'R2 score: {r2*100:2f}%')
print(f'MAE : {mae:5f}')
print(f'MSE : {mse:5f}')

R2 score: 84.904517%
MAE : 0.041336
MSE : 0.004254


### Invers The Metrix...

In [13]:
# Function for invers metrixes...
def invers_prediction(predic):
    prz = predic
    Predictions = []
    for i in range(prz.shape[0]):
        pred = prz[i]*(max_rom-min_rom) + min_rom
        Predictions.append(pred)

    return Predictions


In [14]:
inv_predicted = invers_prediction(predictions_RF)
inver_y_test = invers_prediction(y_test)


### R2 Score for Invers metrixes...

In [16]:
r2 = r2_score(inver_y_test,inv_predicted)
mae = mean_absolute_error(inver_y_test,inv_predicted)
mse = mean_squared_error(inver_y_test,inv_predicted)

print(f'R2 score: {r2*100:2f}%')
print(f'MAE : {mae:5f}')
print(f'MSE : {mse:5f}')

R2 score: 84.904517%
MAE : 2.066800
MSE : 10.635661


In [18]:
pd_result_table = pd.DataFrame(data = [inver_y_test,inv_predicted], index = ["y-test","prd"])
pd_result_table

,0,1,2,3,4,5,6,7,8,9,10,11,12
y-test,138.183863,134.600000,146.660000,131.700000,126.275448,135.000000,130.540507,135.600000,138.932421,121.600000,112.532475,131.340217,140.208569
prd,138.079038,134.144899,139.258933,138.206803,126.540125,130.788035,129.849324,135.533127,139.017755,125.547822,112.614609,133.957734,139.775473
